## Here we will look at how we can evaluate models for a sub-ecosystem of pypi

### The ask is following:

1. Come up with a specific set of stacks, for which we can define a tree.
2. In this example, we will explore the following tree: Web Developer -> Flask -> Flask Admin -> Multiple children

Regarding flask-admin please visit https://github.com/flask-admin/flask-admin

#### Step 1

Collect a few good examples of stacks which use flask-admin.

In [1]:
## Here the stack reflects adding a login functionality to an existing flask admin application using sql as a backend

stack_1 = {"flask", "flask-admin", "flask-SQLAlchemy", "flask-login"}
stack_1_t = stack_1.union({"click", "itsdangerous", "jinja2", "markupsafe", "sqlalchemy", "werkzeug", "wtforms"})

In [2]:
## Here the stack reflects adding a login functionality to an existing flask admin application using mongo as a backend

stack_2 = {"flask", "flask-admin", "flask-mongoengine", "flask-login"}
stack_2_t = stack_2.union({"click", "flask-wtf", "itsdangerous", "jinja2", "markupsafe", "mongoengine", "pymongo", "six", "werkzeug", "wtforms"})

In [3]:
## Here the stack reflects using forms with sql as backend

stack_3 = {"flask", "flask-admin", "flask-SQLAlchemy", "WTForms"}
stack_3_t = stack_3.union({"click", "itsdangerous", "jinja2", "markupsafe", "sqlalchemy", "werkzeug"})

In [4]:
## Here the stack reflects usage of pewee as database backend

stack_4 = {"flask", "flask-admin", "peewee", "wtf-peewee"}
stack_4_t = stack_4.union({"click", "itsdangerous", "jinja2", "markupsafe", "werkzeug", "wtforms"})

#### Step 2

Let's take these stacks as the ground truth and see how the model behaves

- We will first take the intersection of these stacks and find the common packages
- We then input this common stack and see the recommendations of the model. 
- We calculate how precision and recall for the packages which were left out
- We then try to personalize the model with stack specific packages like sqlalchemy and mongoengine and see if the recommendations make sense.

In [212]:
# Let's load the model and the relevant dictionaries

import pickle
import json

with open('HPF_model.pkl', 'rb') as f:
    model = pickle.load(f)

In [213]:
with open('manifest-to-id.pickle', 'rb') as f:
    manifest_to_id_dict = pickle.load(f)

In [214]:
with open('package-to-id-dict.json', 'r') as f:
    package_to_id_dict = json.load(f)

In [215]:
with open('id-to-package-dict.json', 'r') as f:
    id_to_package_dict = json.load(f)

In [174]:
# Let's perform the intersection

stack_base = stack_1.intersection(stack_2).intersection(stack_3).intersection(stack_4)

In [10]:
stack_base

{'flask', 'flask-admin'}

In [11]:
# Let's calculate the remaining packages

stack_remaining = stack_1.union(stack_2).union(stack_3).union(stack_4) - stack_base

In [12]:
stack_remaining

{'WTForms',
 'flask-SQLAlchemy',
 'flask-login',
 'flask-mongoengine',
 'peewee',
 'wtf-peewee'}

In [14]:
from pip._vendor.distlib.util import normalize_name

In [14]:
stack_base = {normalize_name(x) for x in stack_base}

In [15]:
stack_base

{'flask', 'flask-admin'}

In [15]:
stack_remaining = {normalize_name(x) for x in stack_remaining}

NameError: name 'stack_remaining' is not defined

In [17]:
stack_remaining

{'flask-login',
 'flask-mongoengine',
 'flask-sqlalchemy',
 'peewee',
 'wtf-peewee',
 'wtforms'}

In [18]:
# We will start with first step here

stack_base = frozenset(stack_base)

In [19]:
manifest_id = manifest_to_id_dict.get(stack_base)

In [20]:
manifest_id

31606

In [21]:
package_id_list = [package_to_id_dict.get(x) for x in stack_base]

In [22]:
package_id_list

[0, 1084]

In [23]:
# Let's get the recommendations

recommendations = model.topN(user=manifest_id, n=20)

In [24]:
recommendations = [id_to_package_dict.get(str(x)) for x in recommendations]

In [25]:
recommendations

['requests',
 'psycopg2',
 'wsgiref',
 'flask',
 'flask-script',
 'flask-wtf',
 'django',
 'flask-login',
 'gunicorn',
 'nose',
 'mako',
 'python-dateutil',
 'flask-migrate',
 'argparse',
 'selenium',
 'pyyaml',
 'six',
 'pillow',
 'blinker',
 'wheel']

In [26]:
common_package = set(recommendations).intersection(stack_remaining)

In [27]:
# Let's add the common package to stack_base

stack_base = set(stack_base)
stack_base = stack_base.union(common_package)
stack_base = frozenset(stack_base)

In [230]:
stack_base = frozenset(['flask'])

In [231]:
manifest_id = manifest_to_id_dict.get(stack_base)

In [232]:
manifest_id

9205

In [233]:
package_id_list = [package_to_id_dict.get(x) for x in stack_base]

In [234]:
package_id_list

[0]

In [172]:
import pandas as pd
counts_df = pd.DataFrame({
                    'ItemId': package_id_list,
                    'Count': [1] * len(package_id_list)
                })

In [173]:
user_id = model.nusers

In [174]:
model.add_user(
                    user_id=user_id,
                    counts_df=counts_df
                )

True

In [235]:
# Let's get the recommendations

recommendations = model.topN(user=manifest_id, n=20)

In [236]:
recommendations = [id_to_package_dict.get(str(x)) for x in recommendations]

In [237]:
recommendations

['requests',
 'psycopg2',
 'wsgiref',
 'django',
 'gunicorn',
 'nose',
 'python-dateutil',
 'argparse',
 'pyyaml',
 'six',
 'pillow',
 'wheel',
 'pandas',
 'jinja2',
 'ipython',
 'sqlalchemy',
 'flask-sqlalchemy',
 'pycrypto',
 'mock',
 'beautifulsoup4']

#### Here are the observations:

1. We need to train the model by removing the transitives from requirements.txt that we gather. Then and then only, we will start receiving relevant recommendations because if you closely observe the first 10-15 results are mostly transitive dependencies.
2. Need a mechanism to deal with the popular items being recommended. Will be reading further on this.

#### Let's load the model without transitives and check

In [224]:
# Let's load the model and the relevant dictionaries

import pickle
import json

with open('HPF_model_without_trans.pkl', 'rb') as f:
    model = pickle.load(f)

In [225]:
with open('manifest-to-id-without-trans.pickle', 'rb') as f:
    manifest_to_id_dict = pickle.load(f)

In [226]:
with open('package-to-id-dict-without-trans.json', 'r') as f:
    package_to_id_dict = json.load(f)

In [227]:
with open('id-to-package-dict-without-trans.json', 'r') as f:
    id_to_package_dict = json.load(f)

In [228]:
count = 0
l = []
for item_list, user in manifest_to_id_dict.items():
    if 'tensorflow' in item_list and 'keras' in item_list and 'lxml' in item_list:
        count += 1
        l.append(item_list)
print(count)

8


In [229]:
l

[frozenset({'keras', 'lxml', 'tensorflow'}),
 frozenset({'astroml',
            'astropy',
            'bs4',
            'keras',
            'kplr',
            'lxml',
            'pathos',
            'pyfits',
            'rq',
            'scikit-learn',
            'tensorflow',
            'tqdm'}),
 frozenset({'appdirs',
            'flask-cors',
            'gunicorn',
            'keras',
            'lxml',
            'nltk',
            'packaging',
            'pbr',
            'tensorflow',
            'theano'}),
 frozenset({'autocorrect',
            'autoflight',
            'flask',
            'flask-mysqldb',
            'gunicorn',
            'keras',
            'lxml',
            'matplotlib',
            'mysql-python',
            'mysqlclient',
            'nose',
            'pbr',
            'pymysql',
            'pyparsing',
            'python-dateutil',
            'pytz',
            'scikit-learn',
            'sklearn',
            'tensorflow',